## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
import warnings

from src.contract_utils import instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


warnings.filterwarnings('ignore')

ipfs_client = ipfshttpclient.connect()


NUMBER_OF_PARTICIPANTS = 20
NUMBER_OF_ACTIVATED_PARTICIPANTS = 5
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
DISPLAY_TX_EXECUTION = False
TEST_ACCOUNT_ADDRESS = 'bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0'
TEST_ACCOUNT_SEED = 'end spread blind steak install glare pride pony switch exit zone faint ' \
                    'opinion march layer illness can nest fence top debate monitor supreme noble'

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(43)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'temp/root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'temp/proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'temp/root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

### Generate addresses and sign messages

In [2]:
claims_list = []
for i in tqdm(range(NUMBER_OF_PARTICIPANTS)):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

  0%|          | 0/20 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16748806430,QmbQn4imCYtCqmkdEt1kSmrCLUb87UCmd5p3caNxhaXSJ3,0xe8abd810b6ea9b9809b11bdd93b7290bca942730,0x12f8ca97f3153f4dd3860f242dc6a628c33f69fef6cd...,bostrom10c5kc3c0l5jq6xckx8ykwuadynaghye9qzantt,cosmos10c5kc3c0l5jq6xckx8ykwuadynaghye9r3fq4v,sample measure drop fade grape erode upset bus...,bostrom10c5kc3c0l5jq6xckx8ykwuadynaghye9qzantt...,0x2e357cb27b18acbd0669413c879c30e0843d84044a43...,{'pub_key': 'A5W0gSZtVcKdoKz9WTMAAZS3kezkv3JTm...,eyJwdWJfa2V5IjoiQTVXMGdTWnRWY0tkb0t6OVdUTUFBWl...
1,1000000,john16748806431,QmNSQ93hn6vwZHHJbaMfbEgj4NibH6rNY2LJwjavT3hngJ,0xeb63a8eb287ff181e655c7a9318f1af04b1d3d63,0xaf6b104f38211efd0a1b2e927dda46e9469ea0f7ac53...,bostrom1m6sckk9pac80z3der6vu747luhsncxgdarjrdg,cosmos1m6sckk9pac80z3der6vu747luhsncxgd7sxsn0,local lunar ensure sick unfair citizen small d...,bostrom1m6sckk9pac80z3der6vu747luhsncxgdarjrdg...,0x2ded647adc461512ac78d5d8508f82d3fca196208278...,{'pub_key': 'A0rzew97jRef4QQxx49SQRE9q2ovmCsIr...,eyJwdWJfa2V5IjoiQTByemV3OTdqUmVmNFFReHg0OVNRUk...
2,1000000,john16748806432,QmeeR8eFdYyNh2NXCKxtNJpFd7eHuj3yLQdUZ9cXEwpFaq,0xbd2dfeae254034ef5bb913028253b84cfe568dbd,0xfa3020be37091cf53d2e3f7b33821f60d3e5440755c6...,bostrom163ankknvlgykce3a8d6gp4hgvyp5g8ag2hvgn0,cosmos163ankknvlgykce3a8d6gp4hgvyp5g8agfycmdg,salt brick climb crush dash raise flock trap r...,bostrom163ankknvlgykce3a8d6gp4hgvyp5g8ag2hvgn0...,0xbf2de09e2b7b893df515c9d328fb7f3ef52b77f9134f...,{'pub_key': 'Ay8lHI9uF3ieOPOgikeHi9PWZ3jpKEmU7...,eyJwdWJfa2V5IjoiQXk4bEhJOXVGM2llT1BPZ2lrZUhpOV...
3,1000000,john16748806433,QmRXgMj8koWVMWeZ6rr92jUVpyWwwxDBPgDuzzFk2jQQnC,0x9fe16b5b04966ddbff6a544b808507429dc3e0e6,0x7ce62e59f3b464e53a561309adf1d1eac5e56d823178...,bostrom136mj3yrmmad52jvz2x729gh8jmmpv0936qffme,cosmos136mj3yrmmad52jvz2x729gh8jmmpv093ena697,island found calm toss athlete lounge mad leaf...,bostrom136mj3yrmmad52jvz2x729gh8jmmpv0936qffme...,0x76d04b5e4ec507a026288108eefad9b23f219060b2a6...,{'pub_key': 'A/G2IifIIkcj9J9kIwbK5BDvx/R486dns...,eyJwdWJfa2V5IjoiQS9HMklpZklJa2NqOUo5a0l3Yks1Qk...
4,1000000,john16748806434,Qmf96YofNsG2x8WFoz43x2Bc4W2Ur8ecUeMBsd3YHpGs8v,0x2a02c345e00d1c8970bfbe9b25e82d36ac94fdf0,0xc136ebdfa7a74f3e74411841d484f3125fbfbe5116f2...,bostrom140smzep6sxn3cg0fyquctu8e528mr5m3yr3spz,cosmos140smzep6sxn3cg0fyquctu8e528mr5m38s9rl9,tackle witness glide sight vacant hair armor f...,bostrom140smzep6sxn3cg0fyquctu8e528mr5m3yr3spz...,0x84346b0de96c73d4b55542cc3d10cdffb7c22ca810aa...,{'pub_key': 'AmAQ34fvv3Fj2cs7roZ59JHPi0fTNdrQO...,eyJwdWJfa2V5IjoiQW1BUTM0ZnZ2M0ZqMmNzN3JvWjU5Sk...
5,1000000,john16748806435,Qma6GeKs5YYufuZRVBLopx1G9XPPGVr3ArpAaoJZtovLLk,0x8a233f31f47273022d0ff597b7837daed2d7377d,0x2e610c9335f5a42952557ab8bee4fb4988753be21844...,bostrom1vqng698sujsu6kjkaufjsdpyw09lk6kw8necmk,cosmos1vqng698sujsu6kjkaufjsdpyw09lk6kwyqdt93,equip pig toss truck palm resource refuse soda...,bostrom1vqng698sujsu6kjkaufjsdpyw09lk6kw8necmk...,0x03cfc3b9749b5bc5f08970daf4474f5db39de0788581...,{'pub_key': 'A6t+uYHuGrfI+gwrH/RAh39D9KqGPXyzm...,eyJwdWJfa2V5IjoiQTZ0K3VZSHVHcmZJK2d3ckgvUkFoMz...
6,1000000,john16748806436,Qmd65iPvo5Ht7J3j2fibauhDqZWWHg4KDkEikzS2Yt3SiS,0xbbc0a3e0cda63ec7a1abdfd781dc4e427b607860,0xa60dff9600344609c9f81215dcd32b46a22ed264f609...,bostrom15n6vl4t3v52v55xufptwf76937zc2nftaj5yuy,cosmos15n6vl4t3v52v55xufptwf76937zc2nft7pqhzr,curtain flat galaxy auto blade deliver staff p...,bostrom15n6vl4t3v52v55xufptwf76937zc2nftaj5yuy...,0x242efd3af565116d80a2c702858827586880bbeb8e51...,{'pub_key': 'AtdwyfdUM8U+yATp46yFzjB+oiMMRzuWN...,eyJwdWJfa2V5IjoiQXRkd3lmZFVNOFUreUFUcDQ2eUZ6ak...
7,1000000,john16748806437,QmSKB8mPLWT4mB4gc3X1KDuYbKhXycpWrdkxG4mcfNeULj,0xd57534ed8db568b26776c44c26c205ea4523822c,0x360d9b0cb6d3c3f248b96117e360c2654b3c5c58e70b...,bostrom13t5dvpc043kc0lttz8guvsaf6446g0urwl3805,cosmos13t5dvpc043kc0lttz8guvsaf64

### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
print(tasks)
with Pool(processes=NUMBER_OF_THREADS) as pool:
    res = pool.starmap(get_proofs, tasks)
assert res == [True] * len(res)

{('root_testing_source_20_addresses.json', 'temp/proofs_0.json', 0, 41)}
yarn run v1.22.19
$ ts-node index.ts --input ../root_testing_source_20_addresses.json --output ../temp/proofs_0.json --start_index 0 --end_index 41
Merkle root: 6db418e7887fcef5b26a316eff102b1383c6aad9b9c99eea07c763e790d020cf
Number of addresses in the Merkle tree: 41
Done in 1.15s.



In [5]:
roots = []
proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
for task in tasks:
    with open(task[1], 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    roots.append(root_and_proof_json['merkle_root'])
    proofs_df = proofs_df.append(pd.DataFrame(root_and_proof_json['proofs']), ignore_index=True)
assert roots == [roots[0]] * len(roots)
root = roots[0]
proofs_df

,address,amount,proof
0,0xe8abd810b6ea9b9809b11bdd93b7290bca942730,1000000,[b0eb7433037e907edd3ab399706df096cbc779fc73bb5...
1,0xeb63a8eb287ff181e655c7a9318f1af04b1d3d63,1000000,[1e8fa4f111f105217e2f20b6f72f365b69acd020e0051...
2,0xbd2dfeae254034ef5bb913028253b84cfe568dbd,1000000,[358cd72d3ce2faf76c73b71f98deec9f4f6e87e9708df...
3,0x9fe16b5b04966ddbff6a544b808507429dc3e0e6,1000000,[a6cea1297ad5a097d5c20b16fddeee2729dd3497480f0...
4,0x2a02c345e00d1c8970bfbe9b25e82d36ac94fdf0,1000000,[b9bcf3c7b094cab6318cde6f22f762b3b4a489c81d9b6...
5,0x8a233f31f47273022d0ff597b7837daed2d7377d,1000000,[963bc606f808eb1a37ab0e1da9b67bfe51e8a6bf72960...
6,0xbbc0a3e0cda63ec7a1abdfd781dc4e427b607860,1000000,[256007e518308daedaeaef97ce44cbe71c18ee8a8bbde...
7,0xd57534ed8db568b26776c44c26c205ea4523822c,1000000,[78900d7c06c43ae20970087421f80e49257815363f6a3...
8,0x282304cf904faa181e094306f72a9a4fc03f9354,1000000,[1f94f8d147f887d8703011561b94ebff821205cec5384...
9,0x75f76d040ae9b44610bcf22d23fb1e66784441f1,1000000,[70650eb06167ed3c4501b01795488a41e13e41e74966a...


In [6]:
cosmos_proofs_df = proofs_df[proofs_df.address.str.startswith('cosmos')][['address', 'proof']]
ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

claims_with_proofs_df = claims_df\
                .merge(
                    ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
                    how='left',
                    on='ethereum_address')\
                .merge(
                    cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
                    how='left',
                    on='cosmos_address')

claims_with_proofs_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16748806430,QmbQn4imCYtCqmkdEt1kSmrCLUb87UCmd5p3caNxhaXSJ3,0xe8abd810b6ea9b9809b11bdd93b7290bca942730,0x12f8ca97f3153f4dd3860f242dc6a628c33f69fef6cd...,bostrom10c5kc3c0l5jq6xckx8ykwuadynaghye9qzantt,cosmos10c5kc3c0l5jq6xckx8ykwuadynaghye9r3fq4v,sample measure drop fade grape erode upset bus...,bostrom10c5kc3c0l5jq6xckx8ykwuadynaghye9qzantt...,0x2e357cb27b18acbd0669413c879c30e0843d84044a43...,{'pub_key': 'A5W0gSZtVcKdoKz9WTMAAZS3kezkv3JTm...,eyJwdWJfa2V5IjoiQTVXMGdTWnRWY0tkb0t6OVdUTUFBWl...,[b0eb7433037e907edd3ab399706df096cbc779fc73bb5...,[ccb3474484c5086e7d89f9b990dae1a1cdbe63e4c14e4...
1,1000000,john16748806431,QmNSQ93hn6vwZHHJbaMfbEgj4NibH6rNY2LJwjavT3hngJ,0xeb63a8eb287ff181e655c7a9318f1af04b1d3d63,0xaf6b104f38211efd0a1b2e927dda46e9469ea0f7ac53...,bostrom1m6sckk9pac80z3der6vu747luhsncxgdarjrdg,cosmos1m6sckk9pac80z3der6vu747luhsncxgd7sxsn0,local lunar ensure sick unfair citizen small d...,bostrom1m6sckk9pac80z3der6vu747luhsncxgdarjrdg...,0x2ded647adc461512ac78d5d8508f82d3fca196208278...,{'pub_key': 'A0rzew97jRef4QQxx49SQRE9q2ovmCsIr...,eyJwdWJfa2V5IjoiQTByemV3OTdqUmVmNFFReHg0OVNRUk...,[1e8fa4f111f105217e2f20b6f72f365b69acd020e0051...,[ae53c7ee5527fec03a12b3c64058bd0e141ece1166866...
2,1000000,john16748806432,QmeeR8eFdYyNh2NXCKxtNJpFd7eHuj3yLQdUZ9cXEwpFaq,0xbd2dfeae254034ef5bb913028253b84cfe568dbd,0xfa3020be37091cf53d2e3f7b33821f60d3e5440755c6...,bostrom163ankknvlgykce3a8d6gp4hgvyp5g8ag2hvgn0,cosmos163ankknvlgykce3a8d6gp4hgvyp5g8agfycmdg,salt brick climb crush dash raise flock trap r...,bostrom163ankknvlgykce3a8d6gp4hgvyp5g8ag2hvgn0...,0xbf2de09e2b7b893df515c9d328fb7f3ef52b77f9134f...,{'pub_key': 'Ay8lHI9uF3ieOPOgikeHi9PWZ3jpKEmU7...,eyJwdWJfa2V5IjoiQXk4bEhJOXVGM2llT1BPZ2lrZUhpOV...,[358cd72d3ce2faf76c73b71f98deec9f4f6e87e9708df...,[0cc3fb07138e84402e3c0d1aa48b6eab20555233b36b8...
3,1000000,john16748806433,QmRXgMj8koWVMWeZ6rr92jUVpyWwwxDBPgDuzzFk2jQQnC,0x9fe16b5b04966ddbff6a544b808507429dc3e0e6,0x7ce62e59f3b464e53a561309adf1d1eac5e56d823178...,bostrom136mj3yrmmad52jvz2x729gh8jmmpv0936qffme,cosmos136mj3yrmmad52jvz2x729gh8jmmpv093ena697,island found calm toss athlete lounge mad leaf...,bostrom136mj3yrmmad52jvz2x729gh8jmmpv0936qffme...,0x76d04b5e4ec507a026288108eefad9b23f219060b2a6...,{'pub_key': 'A/G2IifIIkcj9J9kIwbK5BDvx/R486dns...,eyJwdWJfa2V5IjoiQS9HMklpZklJa2NqOUo5a0l3Yks1Qk...,[a6cea1297ad5a097d5c20b16fddeee2729dd3497480f0...,[dba658416335dbf7f2897f89937df70c5288c52f67bc1...
4,1000000,john16748806434,Qmf96YofNsG2x8WFoz43x2Bc4W2Ur8ecUeMBsd3YHpGs8v,0x2a02c345e00d1c8970bfbe9b25e82d36ac94fdf0,0xc136ebdfa7a74f3e74411841d484f3125fbfbe5116f2...,bostrom140smzep6sxn3cg0fyquctu8e528mr5m3yr3spz,cosmos140smzep6sxn3cg0fyquctu8e528mr5m38s9rl9,tackle witness glide sight vacant hair armor f...,bostrom140smzep6sxn3cg0fyquctu8e528mr5m3yr3spz...,0x84346b0de96c73d4b55542cc3d10cdffb7c22ca810aa...,{'pub_key': 'AmAQ34fvv3Fj2cs7roZ59JHPi0fTNdrQO...,eyJwdWJfa2V5IjoiQW1BUTM0ZnZ2M0ZqMmNzN3JvWjU5Sk...,[b9bcf3c7b094cab6318cde6f22f762b3b4a489c81d9b6...,[5194d992da3a2ef44b0bbb3616d9530eee355e769f32a...


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [10]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1nn0hd8l3fqhxhsgdeqp3rew79w2nrtm5qedrvamd7a54r2c0czjqj5hcsn


### Initiate Class of Output Parsing

In [11]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [12]:
root_register_output = execute_contract_bash(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        from_address=WALLET_ADDRESS,
                                        contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 3dc8421e3f4b94c8e3aed45c031e95a002007dfac135ca398314b163f15a2cee
Gas used: 121,232
Tx hash: 85B31F8C7895D79C33B890F77B7F87B5F3218E217B4C489D2189575E2C3E34D4


#### Get Merkle Root form the Gift Contract

In [13]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1nn0hd8l3fqhxhsgdeqp3rew79w2nrtm5qedrvamd7a54r2c0czjqj5hcsn
{'data': {'merkle_root': '3dc8421e3f4b94c8e3aed45c031e95a002007dfac135ca398314b163f15a2cee'}}


### Send coins to new addresses

In [14]:
bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 1000
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in bostrom_addresses_chunks:
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=claims_with_proofs_df.bostrom_address.to_list(),
            amounts=[1] * len(claims_with_proofs_df.bostrom_address.to_list()),
            gas=min(71000 * len(claims_with_proofs_df.bostrom_address.to_list()), int(23e6)),
            display_data=DISPLAY_TX_EXECUTION)
    contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 1boot
	receiver: bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q
	amount: 1boot
	receiver: bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf
	amount: 1boot
	receiver: bostrom1efylm4laztlkc0ydye3k6g6cx95h6khw7z27mp
	amount: 1boot
	receiver: bostrom1z4gex65lzw62uqapaxctfku4588ulvgn4yvwmk
	amount: 1boot
	receiver: bostrom13f4sr82dux0kfu7uma3atv7frhzx57ghvm9dxa
	amount: 1boot
	receiver: bostrom1pypc2s9dh0f6yp0xcqwjap99l5k75qgmp2kntw
	amount: 1boot
	receiver: bostrom13ugfw2hr7s8f0yttczg8e2v6v04dlwwarsc4rw
	amount: 1boot
	receiver: bostrom19pfkg9t5nvsmmjffsjv6k625xy98esgukewtyy
	amount: 1boot
	receiver: bostrom1kzrdznpsuhj8scw8p734eex07g0779vr8xw0d6
	amount: 1boot

coin spent
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1boot
	spender: Passport Owner Address
	amount: 1

### Create Passports

In [15]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json = contract_utils.create_passport(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(create_passport_json, row=row)
    else:
        contract_utils.create_passport(row)

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 1413
Gas used: 440,183
Tx hash: CDA96114D1ED58B64B4A2F38BEF6E35C4181F9CD8236B14F039BCF39D2F30C50


### Proof (Add) Address to Passports

In [16]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        proof_ethereum_address_json = contract_utils.proof_address(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(proof_ethereum_address_json, row=row)

        proof_cosmos_address_json = contract_utils.proof_address(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(proof_cosmos_address_json, row=row)
    else:
        contract_utils.proof_address(row)
        contract_utils.proof_address(row, network='cosmos')

0it [00:00, ?it/s]


Events

cyberlinks
	Nickname -> Ethereum Address
	Ethereum Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16495703801
	address: 0x042a2bb0e4acd457bcadda6e05827aa83ebe91fa
Gas used: 281,090
Tx hash: 96C7ADA679DC321B73BF5E090CFA8E086FB906E87B1133E3CD03234F4A568115

Events

cyberlinks
	Nickname -> Cosmos Address
	Cosmos Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16495703801
	address: cosmos162f8clv5ht6j2kmms0556j0mezuj3e6k679fy8
Gas

### Claim

In [17]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        claim_ethereum_json = contract_utils.claim(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(claim_ethereum_json)

        claim_cosmos_json = contract_utils.claim(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(claim_cosmos_json)
    else:
        claim_ethereum_json_2 = contract_utils.claim(row)
        contract_utils.parse_contract_execution_json(claim_ethereum_json_2)

        claim_cosmos_json_2 = contract_utils.claim(row, network='cosmos')
        contract_utils.parse_contract_execution_json(claim_cosmos_json_2)

0it [00:00, ?it/s]

{'claim': {'nickname': 'john16495703800', 'gift_claiming_address': '0x9124d6592264ae9e6693dfa39b8f82f16182419c', 'gift_amount': '1000000', 'proof': ['78900d7c06c43ae20970087421f80e49257815363f6a3a3563f5ec29d28940d1', '8cfc554eaa1feae2ff77f1307b0341fbdf7edf6be643fbe6083fd025f5cc6328', '788ddba1a2fb402628dd12efa2aafbab89e1dfbd57d5c2b279626936a990ba2b', '8fdbf3af68c6049a91547cdc34c7c7d862bc2445ba8e0688a4d334fa48e8fdf6', 'eee157a02965da2c68325270a3893ccad9644d1a717328f173758f04d9d31620']}}

Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x9124d6592264ae9e6693dfa39b8

### Release Gift

In [18]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        release_ethereum_json = contract_utils.release(row, display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(release_ethereum_json)

        release_cosmos_json = contract_utils.release(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(release_cosmos_json)
    else:
        contract_utils.release(row)
        contract_utils.release(row, network='cosmos')

0it [00:00, ?it/s]


Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	sender: Gift Contract
	amount: 1290000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	gift_address: 0x9124d6592264ae9e6693dfa39b8f82f16182419c
	stage: 9
	amount: 1,290,000
Gas used: 157,502
Tx hash: 07B209093AB4960C86A93450B0023387BA9337A3716C92F83BA92D998130DA9B

Events

coin received
	receiver: bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1y90fdf63dz6jfahupu4wdqp6v3f69rxptnr04c wasm /cosmwasm.wasm.v1.MsgEx

## Passport NFT testing
### Transfer Passport

In [19]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [20]:
for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom162f8clv5ht6j2kmms0556j0mezuj3e6ked366q
	recipient: Passport Owner Address
	token_id: 1414
Gas used: 385,801
Tx hash: 5F0B12D48BAB7041AACE2E9EA131AC85065B9D8AEE33D44568A4798D84785BC1


### Burn

In [21]:
for index, row in tqdm(claims_with_proofs_df[2:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1he3vaq8knkx56tevl6mkel42eddq507mgulsnf
	token_id: 1415
Gas used: 381,878
Tx hash: 23476318EFB8D19BDF644F8DFB81BEFF9C0CA9BB188965FEB5E7E7AE76033C5E


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
